## 기준1. 좌표로 거리 구하기       
st_end_daram_df

다람쥐 버스 정류장의 시작점과 종점의 직선거리를 구하여, 기존 다람쥐 버스 노선의 운행 범위를 알아보려고 함.

In [27]:
import sys
import pandas as pd

sys.path.append('./EDA')
sys.path.append('./Modeling')
sys.path.append('./Data/csv')

from EDA_STEP_1 import *
from visualization import *
from clustering import *

In [2]:
st_end_daram_df = get_daram_14_station_df()

In [3]:
start_station = ['111000128','113000113','120000156','120000109','105000127','122000305','123000209']
end_station =   ['111000291','118000048','119000024','120000018','105000072','122000302','123000043']
st_end_node_list = []
for st_end_node in zip(start_station,end_station):
    st_end_node_list.append(st_end_node)
st_end_node_list

[('111000128', '111000291'),
 ('113000113', '118000048'),
 ('120000156', '119000024'),
 ('120000109', '120000018'),
 ('105000127', '105000072'),
 ('122000305', '122000302'),
 ('123000209', '123000043')]

In [4]:
import math
# 하버사인 공식:
## - 지구 곡률을 반영하여 두 지표 사이의 거리를 각도, 호를 사용하여 구하는 방식.
## latitude: 위도(36), longitude: 경도(127)

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


In [5]:
df_node_distance = pd.DataFrame(columns=['BUS_NUM','Radius'])
for bus in st_end_node_list:
    lon1 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lat1 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lon2 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    lat2 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    bus_num = st_end_daram_df['노선명'][st_end_daram_df['NODE_ID']==bus[0]].iloc[0]
    
    radius = haversine_distance(lat1, lon1, lat2, lon2)
    new_data = {'BUS_NUM':bus_num,'Radius':radius}
    df_node_distance = df_node_distance.append(new_data, ignore_index=True)

In [8]:
df_node_distance

,BUS_NUM,Radius
0,8771,2.253933
1,8761,3.270516
2,8551,3.377080
3,8552,2.189844
4,8221,2.017190
5,8441,2.383591
6,8331,4.338116


기준 1 : 모든 다람쥐 버스 노선의 시작점과 종점의 직선거리는 2km이상 4.5km 이내이다.

## 기준 1에 따라 후보군 제거하기

### 1-1. GMM 클러스터된 다람쥐 데이터 가져오기  
GMM 군집결과 총 11개의 클래스.

In [28]:
# gmm 다람쥐 csv 가져오기
gmm_daram_df = pd.read_csv('./Data/csv/gmm_daram.csv', encoding='UTF-8')
gmm_daram_df=gmm_daram_df[['X좌표','Y좌표','NODE_ID', 'gmm_cluster']]
gmm_daram_df

# 다람쥐 95개 정류장 csv 가져오기
daram_df = pd.read_csv('./Data/csv/다람쥐95개정류장.csv', encoding='cp949')

### 1-2.유사도로 뽑은 버스 데이터 가져오기

In [31]:
cosine_daram_df=pd.read_csv('./Data/csv/cosine_similarity_daram.csv', encoding='utf-8')

### 1-3. 95개 다람쥐 버스 정류장에 gmm 군집결과를 labeling 

In [32]:
gmm_daram_result_df = pd.merge(daram_df,gmm_daram_df, left_on='NODE_ID' ,right_on='NODE_ID',how='left')

In [33]:
gmm_daram_result_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'NODE_ID', '정류소명', 'X좌표_x', 'Y좌표_x',
       '법정동코드', '법정동_구', '법정동', 'academy_cnt', 'kindergarten_cnt', 'mart_cnt',
       'restaurant_cnt', 'school_cnt', 'university_cnt', 'subway_cnt',
       'tour_cnt', 'cafe_cnt', 'hospital_cnt', 'culture_cnt',
       'univ_hospital_cnt', 'public_office_cnt', 'employee_cnt', 'alone_ratio',
       'emp_corp_ratio', 'population_15to64', 'RIDE_SUM_6_10',
       'ALIGHT_SUM_6_10', '노선번호', 'X좌표_y', 'Y좌표_y', 'gmm_cluster'],
      dtype='object')

In [34]:
# 불필요한 컬럼 삭제 
gmm_daram_result_df=gmm_daram_result_df.drop(['X좌표_y','Y좌표_y'], axis=1)
gmm_daram_result_df.rename(columns={'X좌표_x':'X좌표','Y좌표_x' :'Y좌표'}, inplace=True)

### 1-4. GMM 클러스터링된 코사인 유사도에서 후보군 제거 

In [35]:
gmm_daram_result_df['구-동']=gmm_daram_result_df['법정동_구']+gmm_daram_result_df['법정동']
# 기존 다람쥐 버스 (95개)가 있는 동을 리스트로 뽑기 
daram_95_gu_list = daram_df['법정동_구'].drop_duplicates().tolist()
daram_95_gu_dong_list = daram_df['법정동'].drop_duplicates().tolist()
daram_95_gu_dong_list

['장안동',
 '답십리동',
 '신천동',
 '방이동',
 '오금동',
 '마천동',
 '수서동',
 '세곡동',
 '자곡동',
 '노량진동',
 '상도동',
 '봉천동',
 '신림동',
 '창전동',
 '노고산동',
 '여의도동',
 '신사동',
 '응암동',
 '구산동',
 '역촌동']

In [40]:
drop_cosine_daram_df = gmm_daram_result_df[~gmm_daram_result_df['구-동'].isin(daram_95_gu_dong_list)]

In [41]:
# 다람쥐 버스가 있던 동 제거
drop_cosine_daram_df = gmm_daram_result_df[~gmm_daram_result_df['구-동'].isin(daram_95_gu_dong_list)]
# 다람쥐 버스가 있던 구 제거
drop_gu_cosine_daram_df = drop_cosine_daram_df[~drop_cosine_daram_df['법정동_구'].isin(daram_95_gu_list)]

### 1-5. 기존 다람쥐 버스 정류장 근처 지역 제거       
새로운 지역에 다람쥐 버스 경로를 신설하기 위해, 기존 다람쥐 버스와 가깝지 않은 정류장들을 선택 해야함. 

In [38]:
# 동이 뭐가 있는지 보기
region_list=drop_cosine_daram_df['법정동'].drop_duplicates().tolist()
region_list

['장안동',
 '답십리동',
 '신천동',
 '방이동',
 '오금동',
 '마천동',
 '수서동',
 '세곡동',
 '자곡동',
 '노량진동',
 '상도동',
 '봉천동',
 '신림동',
 '창전동',
 '노고산동',
 '여의도동',
 '신사동',
 '응암동',
 '구산동',
 '역촌동']

In [42]:
drop_gu_cosine_daram_df

,Unnamed: 0.1,Unnamed: 0,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,academy_cnt,...,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10,노선번호,gmm_cluster,구-동
